Imitation Leaning experiments:
Conclusion: BC works. Others don't seem to be stable even on CartPole.

In [1]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper

In [2]:
env = gym.make("CartPole-v1")
rng = np.random.default_rng(0)

In [3]:
## Behavior Cloning (BC)

In [4]:
rng = np.random.default_rng(0)
env = gym.make("CartPole-v1")
expert = PPO(policy=MlpPolicy, env=env)
expert.learn(10000)

In [5]:
reward, _ = evaluate_policy(
    expert.policy,  # type: ignore[arg-type]
    env,
    n_eval_episodes=1,
    render=False,
)
print(f"Reward of expert: {reward}")

Reward of expert: 308.0


In [6]:
rollouts = rollout.rollout(
    expert,
    DummyVecEnv([lambda: RolloutInfoWrapper(env)]),
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),
    rng=rng,
)

In [7]:
transitions = rollout.flatten_trajectories(rollouts)

In [8]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [9]:
bc_trainer.train(n_epochs=10)

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 36.5      |
|    loss           | 0.693     |
|    neglogp        | 0.694     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


103batch [00:00, 1027.63batch/s]
353batch [00:00, 1194.67batch/s]
473batch [00:00, 1181.52batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000621 |
|    entropy        | 0.621     |
|    epoch          | 2         |
|    l2_loss        | 0         |
|    l2_norm        | 42.1      |
|    loss           | 0.615     |
|    neglogp        | 0.616     |
|    prob_true_act  | 0.57      |
|    samples_so_far | 16032     |
---------------------------------


595batch [00:00, 1194.67batch/s]
845batch [00:00, 1224.23batch/s]
968batch [00:00, 1211.46batch/s]

...

1090batch [00:00, 1212.40batch/s]
1343batch [00:01, 1238.64batch/s]
1467batch [00:01, 1237.81batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 1500     |
|    ent_loss       | -0.00063 |
|    entropy        | 0.63     |
|    epoch          | 6        |
|    l2_loss        | 0        |
|    l2_norm        | 42.9     |
|    loss           | 0.564    |
|    neglogp        | 0.564    |
|    prob_true_act  | 0.591    |
|    samples_so_far | 48032    |
--------------------------------



1719batch [00:01, 1245.14batch/s]
1974batch [00:01, 1256.86batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 2000      |
|    ent_loss       | -0.000615 |
|    entropy        | 0.615     |
|    epoch          | 8         |
|    l2_loss        | 0         |
|    l2_norm        | 43.1      |
|    loss           | 0.559     |
|    neglogp        | 0.56      |
|    prob_true_act  | 0.597     |
|    samples_so_far | 64032     |
---------------------------------



2226batch [00:01, 1248.07batch/s]
2270batch [00:01, 1222.35batch/s]


In [10]:
reward, _ = evaluate_policy(bc_trainer.policy, env, 50)
print("Reward:", reward)

Reward: 252.56


In [30]:
#### GAIL algorithm - train a PPO to fool a discriminator into thinking the trajectory is expert

In [11]:
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
import seals

In [12]:
env = gym.make("seals/CartPole-v0")
expert = PPO(
    policy=MlpPolicy,
    env=env,
    seed=0,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
    n_steps=64,
)
expert.learn(20000)

In [13]:
rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    make_vec_env(
        "seals/CartPole-v0",
        n_envs=5,
        post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],
        rng=rng,
    ),
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=rng,
)

In [14]:
venv = make_vec_env("seals/CartPole-v0", n_envs=8, rng=rng)
learner = PPO(
    env=venv,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
)
reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=2048,
    n_disc_updates_per_round=4,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net
)

In [15]:
learner_rewards_before_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True
)
print(np.array(learner_rewards_before_training).sum()/len(learner_rewards_before_training))

8.31


In [16]:
gail_trainer.train(300000)  # Note: set to 300000 for better results

round:   0%|                                             | 0/18 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 500      |
|    gen/rollout/ep_rew_mean  | 35.8     |
|    gen/time/fps             | 16474    |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 0        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.431    |
|    disc/disc_acc_expert             | 0.231    |
|    disc/disc_acc_gen                | 0.63     |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.702    |
|    disc/disc_proportion_expert_pred | 0.301    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   6%|██                                   | 1/18 [00:03<00:56,  3.35s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 30.2        |
|    gen/rollout/ep_rew_wrapped_mean | 327         |
|    gen/time/fps                    | 18377       |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 0           |
|    gen/time/total_timesteps        | 32768       |
|    gen/train/approx_kl             | 0.008570074 |
|    gen/train/clip_fraction         | 0.0254      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.688      |
|    gen/train/explained_variance    | 0.0455      |
|    gen/train/learning_rate         | 0.0003      |
|    gen/train/loss                  | 0.614       |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.00226    |
|    gen/train/value_loss            | 16.5   

round:  11%|████                                 | 2/18 [00:06<00:52,  3.27s/it]

...

round:  89%|████████████████████████████████    | 16/18 [00:52<00:06,  3.27s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 48.8        |
|    gen/rollout/ep_rew_wrapped_mean | 332         |
|    gen/time/fps                    | 18174       |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 0           |
|    gen/time/total_timesteps        | 278528      |
|    gen/train/approx_kl             | 0.004511253 |
|    gen/train/clip_fraction         | 0.0412      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.526      |
|    gen/train/explained_variance    | 0.171       |
|    gen/train/learning_rate         | 0.0003      |
|    gen/train/loss                  | 30.7        |
|    gen/train/n_updates             | 160         |
|    gen/train/policy_gradient_loss  | -0.000516   |
|    gen/train/value_loss            | 49     

round:  94%|██████████████████████████████████  | 17/18 [00:55<00:03,  3.27s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 51.7         |
|    gen/rollout/ep_rew_wrapped_mean | 315          |
|    gen/time/fps                    | 18212        |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 0            |
|    gen/time/total_timesteps        | 294912       |
|    gen/train/approx_kl             | 0.0038500326 |
|    gen/train/clip_fraction         | 0.0391       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.508       |
|    gen/train/explained_variance    | 0.176        |
|    gen/train/learning_rate         | 0.0003       |
|    gen/train/loss                  | 41.4         |
|    gen/train/n_updates             | 170          |
|    gen/train/policy_gradient_loss  | -0.00133     |
|    gen/train/value_loss   

round: 100%|████████████████████████████████████| 18/18 [00:58<00:00,  3.27s/it]


In [17]:
learner_rewards_after_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True
)
print(np.array(learner_rewards_after_training).sum()/len(learner_rewards_after_training))

66.96


In [18]:
learner_rewards_after_training

[89.0,
 66.0,
 56.0,
 57.0,
 85.0,
 100.0,
 91.0,
 64.0,
 62.0,
 62.0,
 41.0,
 67.0,
 76.0,
 86.0,
 56.0,
 61.0,
 67.0,
 57.0,
 55.0,
 58.0,
 52.0,
 66.0,
 80.0,
 58.0,
 82.0,
 35.0,
 81.0,
 56.0,
 33.0,
 60.0,
 70.0,
 119.0,
 42.0,
 71.0,
 60.0,
 21.0,
 73.0,
 86.0,
 64.0,
 58.0,
 48.0,
 47.0,
 30.0,
 56.0,
 82.0,
 81.0,
 45.0,
 74.0,
 77.0,
 72.0,
 71.0,
 80.0,
 108.0,
 90.0,
 111.0,
 81.0,
 65.0,
 85.0,
 61.0,
 67.0,
 57.0,
 59.0,
 86.0,
 68.0,
 55.0,
 66.0,
 127.0,
 71.0,
 59.0,
 68.0,
 67.0,
 24.0,
 53.0,
 83.0,
 71.0,
 61.0,
 70.0,
 62.0,
 30.0,
 73.0,
 59.0,
 96.0,
 54.0,
 38.0,
 57.0,
 66.0,
 31.0,
 58.0,
 69.0,
 72.0,
 60.0,
 65.0,
 84.0,
 86.0,
 86.0,
 87.0,
 71.0,
 53.0,
 94.0,
 46.0]

In [ ]:
#### COPY PASTE OF GAIL CODE AGAIN

In [ ]:
import numpy as np
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms.adversarial.gail import GAIL
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env

rng = np.random.default_rng(0)

env = gym.make("seals/CartPole-v0")
expert = PPO(policy=MlpPolicy, env=env, n_steps=64)
expert.learn(1000)

rollouts = rollout.rollout(
    expert,
    make_vec_env(
        "seals/CartPole-v0",
        n_envs=5,
        post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],
        rng=rng,
    ),
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=rng,
)

venv = make_vec_env("seals/CartPole-v0", n_envs=8, rng=rng)
learner = PPO(env=venv, policy=MlpPolicy)
reward_net = BasicRewardNet(
    venv.observation_space,
    venv.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=2048,
    n_disc_updates_per_round=4,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
)

gail_trainer.train(20000)
rewards, _ = evaluate_policy(learner, venv, 100, return_episode_rewards=True)
print("Rewards:", rewards)

In [ ]:
### AIRL CODE PASTE TO TRY

In [20]:
import numpy as np
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms.adversarial.airl import AIRL
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
import seals

rng = np.random.default_rng(0)

env = gym.make("seals/CartPole-v0")
expert = PPO(policy=MlpPolicy, env=env)
expert.learn(50000)

rollouts = rollout.rollout(
    expert,
    make_vec_env(
        "seals/CartPole-v0",
        rng=rng,
        n_envs=5,
        post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],
    ),
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=rng,
)

venv = make_vec_env("seals/CartPole-v0", rng=rng, n_envs=8)
learner = PPO(env=venv, policy=MlpPolicy)
reward_net = BasicShapedRewardNet(
    venv.observation_space,
    venv.action_space,
    normalize_input_layer=RunningNorm,
)
airl_trainer = AIRL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=2048,
    n_disc_updates_per_round=4,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
)
airl_trainer.train(200000)
rewards, _ = evaluate_policy(learner, venv, 100, return_episode_rewards=True)
print("Rewards:", rewards)

round:   0%|                                             | 0/12 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 500      |
|    gen/rollout/ep_rew_mean  | 29.8     |
|    gen/time/fps             | 17021    |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 0        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.5      |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0        |
|    disc/disc_entropy                | 0.602    |
|    disc/disc_loss                   | 0.806    |
|    disc/disc_proportion_expert_pred | 1        |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   8%|███                                  | 1/12 [00:03<00:36,  3.36s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 25.3        |
|    gen/rollout/ep_rew_wrapped_mean | 354         |
|    gen/time/fps                    | 16997       |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 0           |
|    gen/time/total_timesteps        | 32768       |
|    gen/train/approx_kl             | 0.009204772 |
|    gen/train/clip_fraction         | 0.0408      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.685      |
|    gen/train/explained_variance    | -0.0582     |
|    gen/train/learning_rate         | 0.0003      |
|    gen/train/loss                  | 3.06        |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.003      |
|    gen/train/value_loss            | 88.8   

round:  17%|██████▏                              | 2/12 [00:06<00:33,  3.33s/it]

...

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 10.6         |
|    gen/rollout/ep_rew_wrapped_mean | 27.6         |
|    gen/time/fps                    | 17109        |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 0            |
|    gen/time/total_timesteps        | 180224       |
|    gen/train/approx_kl             | 0.0021357045 |
|    gen/train/clip_fraction         | 0.0223       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.28        |
|    gen/train/explained_variance    | 0.919        |
|    gen/train/learning_rate         | 0.0003       |
|    gen/train/loss                  | 0.571        |
|    gen/train/n_updates             | 100          |
|    gen/train/policy_gradient_loss  | -0.000156    |
|    gen/train/value_loss   

round:  92%|█████████████████████████████████   | 11/12 [00:36<00:03,  3.30s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 10.5         |
|    gen/rollout/ep_rew_wrapped_mean | -36.8        |
|    gen/time/fps                    | 17012        |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 0            |
|    gen/time/total_timesteps        | 196608       |
|    gen/train/approx_kl             | 0.0032767793 |
|    gen/train/clip_fraction         | 0.0195       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.252       |
|    gen/train/explained_variance    | 0.908        |
|    gen/train/learning_rate         | 0.0003       |
|    gen/train/loss                  | 0.634        |
|    gen/train/n_updates             | 110          |
|    gen/train/policy_gradient_loss  | -0.00102     |
|    gen/train/value_loss   

round: 100%|████████████████████████████████████| 12/12 [00:39<00:00,  3.32s/it]


Rewards: [7.0, 8.0, 9.0, 8.0, 8.0, 9.0, 9.0, 7.0, 8.0, 9.0, 7.0, 8.0, 9.0, 7.0, 8.0, 8.0, 8.0, 9.0, 9.0, 8.0, 8.0, 9.0, 8.0, 10.0, 9.0, 8.0, 8.0, 8.0, 7.0, 9.0, 7.0, 8.0, 7.0, 9.0, 8.0, 9.0, 9.0, 8.0, 9.0, 9.0, 9.0, 9.0, 9.0, 8.0, 7.0, 9.0, 8.0, 7.0, 9.0, 9.0, 9.0, 9.0, 7.0, 8.0, 8.0, 8.0, 7.0, 10.0, 8.0, 8.0, 9.0, 9.0, 9.0, 8.0, 7.0, 8.0, 9.0, 7.0, 7.0, 8.0, 9.0, 8.0, 9.0, 8.0, 7.0, 9.0, 7.0, 9.0, 10.0, 8.0, 8.0, 8.0, 8.0, 8.0, 7.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 7.0, 8.0, 9.0, 9.0, 9.0, 8.0, 8.0, 9.0, 9.0]
